** Step 1. Set up emulab geni-lib package for CloudLab **

In [ ]:
!unzip codes/cloudlab/emulab-0.9.zip -d codes/cloudlab

In [ ]:
!cd codes/cloudlab/emulab-geni-lib-1baf79cf12cb/;\
    module purge;\
    module load anaconda/4.2.0;\
    python setup.py install --user

In [1]:
!ls /home/lngo/.local/lib/python2.7/site-packages/

dsml.py			      pdfminer-20140328.dist-info
dsml.pyc		      pkg_resources
dsml.pyo		      plotly
easy-install.pth	      plotly-1.9.6.dist-info
easy_install.py		      protobuf-3.0.0b2.dist-info
easy_install.pyc	      protobuf-3.0.0b2-py2.7-nspkg.pth
examples		      pydoop
external		      pydoop-1.2.0.dist-info
feedparser-5.2.1.dist-info    python_ldap-2.4.25.dist-info
feedparser.py		      remote_ikernel
feedparser.pyc		      remote_ikernel-0.4.5.dist-info
filechunkio		      requests-2.13.0-py2.7.egg
filechunkio-1.6.dist-info     requests_oauthlib
geni_lib-0.9.4.6-py2.7.egg    requests_oauthlib-0.5.0.dist-info
google			      seal
happybase		      seal-devel_bf3090f-py2.7.egg-info
happybase-0.9.dist-info       setuptools
kafka			      setuptools-23.1.0.dist-info
kafka_python-1.3.1.dist-info  setuptools.pth
ldap			      six-1.10.0.dist-info
_ldap.so		      six.py
ldapurl.py		      six.pyc
ldapurl.pyc		      tensorflow
ldapurl.pyo		      tensorflow-0.9.0.dist-info
ldif.py			      thrift
ldi

In [ ]:
!rm -Rf codes/cloudlab/emulab-geni-lib-1baf79cf12cb/

** Step 2: Reload geni-lib for the first time **

On the top bar of this notebook, select `Kernel` and then `Restart`

In [1]:
%%writefile codes/cloudlab/xenvm.py

"""An example of constructing a profile with a single Xen VM from CloudLab.

Instructions:
Wait for the profile instance to start, and then log in to the VM via the
ssh port specified below.  (Note that in this case, you will need to access
the VM through a high port on the physical host, since we have not requested
a public IP address for the VM itself.)
"""

import geni.portal as portal
import geni.rspec.pg as rspec

# Create a Request object to start building the RSpec.
request = portal.context.makeRequestRSpec()
# Create a XenVM
node = request.XenVM("node")
node.disk_image = "urn:publicid:IDN+emulab.net+image+emulab-ops:UBUNTU14-64-STD"
node.routable_control_ip = "true"

node.addService(rspec.Execute(shell="/bin/sh",
                              command="sudo apt-get update"))
node.addService(rspec.Execute(shell="/bin/sh",
                              command="sudo apt-get install -y apache2"))
# Print the RSpec to the enclosing page.
portal.context.printRequestRSpec()

Overwriting codes/cloudlab/xenvm.py


** Step 3: Test emulab geni-lib installation **

Executing the cell below should produce an XML element with the following content:

```
<rspec xmlns:client="http://www.protogeni.net/resources/rspec/ext/client/1" xmlns:emulab="http://www.protogeni.net/resources/rspec/ext/emulab/1" xmlns:jacks="http://www.protogeni.net/resources/rspec/ext/jacks/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.geni.net/resources/rspec/3" xsi:schemaLocation="http://www.geni.net/resources/rspec/3 http://www.geni.net/resources/rspec/3/request.xsd" type="request">
  <rspec_tour xmlns="http://www.protogeni.net/resources/rspec/ext/apt-tour/1">
    <description type="markdown">An example of constructing a profile with a single Xen VM.</description>
    <instructions type="markdown">Wait for the profile instance to start, and then log in to the VM via the
ssh port specified below.  (Note that in this case, you will need to access
the VM through a high port on the physical host, since we have not requested
a public IP address for the VM itself.)
</instructions>
  </rspec_tour>
  <node client_id="node" exclusive="false">
    <sliver_type name="emulab-xen"/>
  </node>
</rspec>
```

In [2]:
!module purge;\
    module load anaconda/4.2.0;\
    python codes/cloudlab/xenvm.py

<rspec xmlns:client="http://www.protogeni.net/resources/rspec/ext/client/1" xmlns:emulab="http://www.protogeni.net/resources/rspec/ext/emulab/1" xmlns:jacks="http://www.protogeni.net/resources/rspec/ext/jacks/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.geni.net/resources/rspec/3" xsi:schemaLocation="http://www.geni.net/resources/rspec/3 http://www.geni.net/resources/rspec/3/request.xsd" type="request">
  <rspec_tour xmlns="http://www.protogeni.net/resources/rspec/ext/apt-tour/1">
    <description type="markdown">An example of constructing a profile with a single Xen VM from CloudLab.</description>
    <instructions type="markdown">Wait for the profile instance to start, and then log in to the VM via the
ssh port specified below.  (Note that in this case, you will need to access
the VM through a high port on the physical host, since we have not requested
a public IP address for the VM itself.)
</instructions>
  </rspec_tour>
  <node client_id="node" exclus

** If things fail, where to go? **

```
/var/tmp/start-up*
```

- The geni-lib library helps validate the syntatical correctness of your python-based experimental design
- It is similar to Amazon AWS' [Cloud Formation](https://aws.amazon.com/cloudformation/)
- It does not help validate the structural correctness of your experiment upon deployment
- Manual modifications of design codes and experimental profiles are time-consuming
- CloudLab supports linking to experimental codes located on Github, and the experimental profiles will automatically update via git's web-hooks

- Fork the following github repository: https://github.com/lbstoller/my-profile into your Github account
- Create a new experimental profile that points to the forked repository in your account
- Setup web-hook
- Instantiate one experiment using the original version of the profile
- Modify the `profile.py` that it uses the **UNTU14-64-STD** image and has a public IP interface
- Modify the `silly.sh` file so that it contains the command to update the package and installs apache2
- Instantiate a second experiment using the profile
- Confirm that the two experiments are different

** Example of designing an experimental profile for Cloud Computing infrastructure on CloudLab using geni-lib **

- Sets the template for future assignments in this class
- Is based on CloudLab's default stack profile
- https://github.com/linhbngo/cloudlab_openstack
